##파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

In [ ]:
from glob import glob
data = pd.read_json('/content/drive/MyDrive/kubig/NLP/train_original.json')
data2 = pd.read_json('/content/drive/MyDrive/kubig/NLP/valid_original.json')

In [ ]:
def preprocess(data):
    csv = []
    for i in tqdm(range(len(data['documents']))):
        news = data['documents'][i]
        id = news['id']
        category = news['category']
        media_type = news['media_type']
        media_sub_type = news['media_sub_type']
        media_name = news['media_name']
        size = news['size']
        char_count = news['char_count']
        date = news['publish_date']
        title = news['title']
        texts = ''
        for idx, line in enumerate(news['text']):
            try:
                text = line[0]['sentence']
                texts += text
                texts += ' '
            except:
                None
        ext = news['extractive']
        summary = ''
        for j in ext:
            try:
                line = news['text'][j][0]['sentence']
                summary += line
                summary += ' '
            except:
                None
        abs = news['abstractive'][0]
        csv.append([id, category, media_type, media_sub_type, media_name, size, char_count, date, title, texts, summary, abs])
    return csv

In [ ]:
csv = preprocess(data)
csv2 = preprocess(data2)

100%|██████████| 30122/30122 [00:00<00:00, 49076.52it/s]


In [ ]:
df = pd.DataFrame(csv, columns=['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size', 'char_count', 'date', 'title', 'news', 'summary', 'gen'])
sci = df[df['category']=='IT,과학'].reset_index()

In [ ]:
df2 = pd.DataFrame(csv2, columns=['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size', 'char_count', 'date', 'title', 'news', 'summary', 'gen'])
sci2 = df2[df2['category']=='IT,과학'].reset_index()

In [ ]:
sci = sci[['id', 'date', 'title', 'news', 'summary']]
sci2 = sci2[['id', 'date', 'title', 'news', 'summary']]

In [ ]:
for i, line in enumerate(sci2['summary']):
    if len(line) <=20:
        sci2 = sci2.drop(i, axis=0)
for i, line in enumerate(sci2['news']):
    if len(line) <=50:
        sci2 = sci2.drop(i, axis=0)

for i, line in enumerate(sci['summary']):
    if len(line) <=20:
        sci = sci.drop(i, axis=0)
for i, line in enumerate(sci['news']):
    if len(line) <=50:
        sci = sci.drop(i, axis=0)

In [ ]:
#import re

#def review_to_words(text):
#    text = re.sub(r'[^가-힣0-9]', ' ', text)
#    return text

In [ ]:
#sci['news'] = sci['news'].apply(review_to_words)


In [ ]:
sci[:3]

,id,date,title,news,summary
0,358249366,2019-09-01 00:09:00,글로벌 스마트폰 판매 하락 속 삼성 `선전`...20%대 점유율 돌파,올해 2분기 전 세계 스마트폰 판매량은 하락세를 기록했으나 삼성전자는 6분기 연속 ...,올해 2분기 전 세계 스마트폰 판매량은 하락세를 기록했으나 삼성전자는 6분기 연속 ...
1,358256453,2019-09-01 09:00:00,"LGU+, 장애가정 청소년 자산형성 돕는 `두드림 U+요술통장` 대상자 모집",LG유플러스는 한국장애인재활협회와 함께 장애가정 청소년들의 자산형성을 돕는 '두드림...,LG유플러스는 한국장애인재활협회와 함께 장애가정 청소년들의 자산형성을 돕는 '두드림...
2,358256455,2019-09-01 09:00:00,"LGU+, 장애가정 청소년 지원 ‘두드림 요술통장’ 신규 대상자 모집",지난 7월 서울 롯데월드에서 진행된 '2019 두드림 U+요술통장 여름캠프' 참가자...,LG유플러스와 한국장애인재활협회가 장애가정 청소년들의 자산형성을 돕는 '두드림 U+...


#모델

In [ ]:
model = EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")

pytorch_model.bin:   0%|          | 0.00/589M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
import pandas as pd

#모델

In [ ]:
tokenizer =BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/kiyoungkim1/LMkor
!pip3 install -q transformers

from LMkor.examples.bertshared_summarization import Summarize
summarize = Summarize('kykim/bertshared-kor-base')

Cloning into 'LMkor'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 101 (delta 17), reused 11 (delta 11), pack-reused 80
Receiving objects: 100% (101/101), 45.28 KiB | 662.00 KiB/s, done.
Resolving deltas: 100% (46/46), done.


pytorch_model.bin:   0%|          | 0.00/589M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
from transformers import BertTokenizerFast, EncoderDecoderModel
model_bertshared = EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")

In [ ]:
random = sci.sample(n=10, random_state=40)

In [ ]:
random

,id,date,title,news,summary
959,358401759,2019-09-02 14:55:48,"쾌적 수면을 돕는 프리미엄 온수매트, 나비엔 메이트 신제품 출시",경동나비엔이 쾌적한 수면을 돕는 프리미엄 온수매트 '나비엔 메이트' 신제품을 출시한...,2015년 첫 선을 보인 나비엔 메이트는 온도제어 기술력을 적용해 만들어 매트 전체...
793,356000578,2019-08-12 06:01:07,음성인식 가능한 똑똑한 안마의자 `오레스트 OR-8000`,[기자가 체험한 중소기업 상품 톡톡-17] 최근 안마의자가 단순히 효도상품이 아닌 ...,오레스트 OR-8000의 또 다른 장점은 통상 4개 정도만 있는 마사지 볼이 8개나...
1055,359634594,2019-09-11 16:24:51,"대웅제약, 현업 부서가 직접 인재 선발하는 상시 공개채용 도입",대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는...,대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는...
1597,332732267,2019-01-30 11:00:00,"과기정통부-조달청, 'R&D 성과 공공조달 연계 위한 업무협약' 체결",과학기술정보통신부와 조달청은 30일 조달청에서 '과학기술 연구개발 성과와 공공조달 ...,과학기술정보통신부와 조달청은 30일 조달청에서 '과학기술 연구개발 성과와 공공조달 ...
158,364650956,2019-10-26 10:57:00,"아이폰11 첫날 개통량, 전작보다 30% 늘었다",지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 XS·XR시리...,지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 XS·XR시리...
1779,334116374,2019-02-14 08:46:00,"NHN엔터, 지난해 영업익 687억..'1조 클럽' 달성","NHN엔터테인먼트(181710)는 지난해 4분기 연결기준 매출 3858억원, 영업이...","NHN엔터테인먼트(181710)는 지난해 4분기 연결기준 매출 3858억원, 영업이..."
1388,331133400,2019-01-16 10:37:00,"네이버, 사용자 취향 분석해 쇼핑 검색결과 제공",네이버가 인공지능(AI)기반 상품 추천 시스템 에이아이템즈(AiTEMS)를 활용해 ...,네이버가 인공지능(AI)기반 상품 추천 시스템 에이아이템즈(AiTEMS)를 활용해 ...
747,355261054,2019-08-05 14:55:17,"맵퍼스, 도로공사와 고속도로 사고 대응 MOU",전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발...,전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발...
739,355048278,2019-08-02 15:14:10,순천향대서울병원 `Best of the Best` 인정,서유성 순천향대서울병원장 순천향대학교 서울병원(원장 서유성)이 최근 보건복지부가 발...,순천향대 서울병원은 △의료질과 환자안전 △의료전달체계 △공공성 평가 영역에서 1등급...
272,368093505,2019-11-22 09:09:00,공공의대 2023년 개교하나?...공청회서 찬반 공세 예고,의료취약지와 공공보건 분야에서 복무할 의사를 양성할 국립공공보건의료대학 설립 법안에...,의료취약지와 공공보건 분야에서 복무할 의사를 양성할 국립공공보건의료대학 설립 법안에...


In [ ]:
print(random['summary'])

959     2015년 첫 선을 보인 나비엔 메이트는 온도제어 기술력을 적용해 만들어 매트 전체...
793     오레스트 OR-8000의 또 다른 장점은 통상 4개 정도만 있는 마사지 볼이 8개나...
1055    대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는...
1597    과학기술정보통신부와 조달청은 30일 조달청에서 '과학기술 연구개발 성과와 공공조달 ...
158     지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 XS·XR시리...
1779    NHN엔터테인먼트(181710)는 지난해 4분기 연결기준 매출 3858억원, 영업이...
1388    네이버가 인공지능(AI)기반 상품 추천 시스템 에이아이템즈(AiTEMS)를 활용해 ...
747     전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발...
739     순천향대 서울병원은 △의료질과 환자안전 △의료전달체계 △공공성 평가 영역에서 1등급...
272     의료취약지와 공공보건 분야에서 복무할 의사를 양성할 국립공공보건의료대학 설립 법안에...
Name: summary, dtype: object


In [ ]:
result = random['news'].apply(lambda x: summarize(x))

경동나비엔이 쾌적한 수면환경을 제공하는 프리미엄 온수매트 나비엔 메이트 신제품을 출시한다.
안마의자가 단순히 효도상품이 아닌 필수품으로 각광받는 가운데, 최근 오레스트 매장에는 마사지 볼이 8개나 있어 더욱 효과적으로 안마를 받을 수 있다.
대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는 상시 공개 채용 방식을 도입해 새로운 채용 방식에 따라 각 직무 별 역량을 갖춘 인재를 확보할 계획이다.
과학기술정보통신부와 조달청은 30일 연구개발 성과가 국민 생활 속에서 실현되는 길을 확장하도록 하는데 양 기관이 뜻을 같이하여 과학기술 연구개발 성과를 높이라고 하는 업무협약을 체결했다고 전했다고 밝혔으며, 이번에 체결한 협약은 연구개발 결과물 성능화, 혁신적 제품의 사업화를 지원하기 위한 신규 사업을 착수하는 것이다.
지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 xs xr시리즈보다 30 % 가량 높은 것으로 나타났는데 이는 국내 이통사들의 5g 품질에 만족하지 못하는 고객이 많기 때문으로 보인다.
nhn엔터테인먼트 ( 1817 ) 는 기존 게임사업의 체질 개선과 신사업의 확대로 2013년 8월 분할 이후 연간 기준으로 사상 첫 매출액 1조를 돌파했고, 영업이익 또한 꾸준히 상승하고 있다고 14일 공시했다.
네이버는 인공지능 ( ai ) 기반 상품 추천 시스템 에이아이템즈를 활용해 검색 결과에서도 관심사 선호도에 따라 쇼핑을 경험할 수 있는 포유 ( for you ) 영역을 신설했다.
전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발을 위한 양해각서 ( mou ) 를 체결했다고 밝히며 아틀란 내비게이션 앱의 속도정보를 통해 감지한 고속도로 상의 돌발상황을 도로공사에 실시간으로 전송하고, 폐쇄회로tv를 통해 이를 확인하고 신속하게 상황을
순천향대 서울병원이 전국 337개 상급종합병원과 종합병원을 대상으로 실시한 2019 의료질평가 에서 1등급 중에서도 최고 등급인 1 - 가 등급을

In [ ]:
result = ['경동나비엔이 쾌적한 수면환경을 제공하는 프리미엄 온수매트 나비엔 메이트 신제품을 출시한다.',
          '안마의자가 단순히 효도상품이 아닌 필수품으로 각광받는 가운데, 최근 오레스트 매장에는 마사지 볼이 8개나 있어 더욱 효과적으로 안마를 받을 수 있다.',
'대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는 상시 공개 채용 방식을 도입해 새로운 채용 방식에 따라 각 직무 별 역량을 갖춘 인재를 확보할 계획이다.',
'과학기술정보통신부와 조달청은 30일 연구개발 성과가 국민 생활 속에서 실현되는 길을 확장하도록 하는데 양 기관이 뜻을 같이하여 과학기술 연구개발 성과를 높이라고 하는 업무협약을 체결했다고 전했다고 밝혔으며, 이번에 체결한 협약은 연구개발 결과물 성능화, 혁신적 제품의 사업화를 지원하기 위한 신규 사업을 착수하는 것이다.',
'지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 xs xr시리즈보다 30 % 가량 높은 것으로 나타났는데 이는 국내 이통사들의 5g 품질에 만족하지 못하는 고객이 많기 때문으로 보인다.',
'nhn엔터테인먼트 ( 1817 ) 는 기존 게임사업의 체질 개선과 신사업의 확대로 2013년 8월 분할 이후 연간 기준으로 사상 첫 매출액 1조를 돌파했고, 영업이익 또한 꾸준히 상승하고 있다고 14일 공시했다.',
'네이버는 인공지능 ( ai ) 기반 상품 추천 시스템 에이아이템즈를 활용해 검색 결과에서도 관심사 선호도에 따라 쇼핑을 경험할 수 있는 포유 ( for you ) 영역을 신설했다.',
'전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발을 위한 양해각서 ( mou ) 를 체결했다고 밝히며 아틀란 내비게이션 앱의 속도정보를 통해 감지한 고속도로 상의 돌발상황을 도로공사에 실시간으로 전송하고, 폐쇄회로tv를 통해 이를 확인하고 신속하게 상황을',
'순천향대 서울병원이 전국 337개 상급종합병원과 종합병원을 대상으로 실시한 2019 의료질평가 에서 1등급 중에서도 최고 등급인 1 - 가 등급을 받았다.',
'22일 국회 보건복지위원회는 공청회를 열어 의료취약지와 공공보건 분야에서 복무할 의사를 양성할 국립공공보건의료대학 설립 법안에 대한 찬반 의견을 듣는데, 이에 대해 임준 서울시립대 교수와 조승연 지방의료원연합회장, 안덕선 대한의사협회 의료정책연구소장이 진술인으로 나온다.']


In [ ]:
random['result']=result

In [ ]:
random

,id,date,title,news,summary,result
959,358401759,2019-09-02 14:55:48,"쾌적 수면을 돕는 프리미엄 온수매트, 나비엔 메이트 신제품 출시",경동나비엔이 쾌적한 수면을 돕는 프리미엄 온수매트 '나비엔 메이트' 신제품을 출시한...,2015년 첫 선을 보인 나비엔 메이트는 온도제어 기술력을 적용해 만들어 매트 전체...,경동나비엔이 쾌적한 수면환경을 제공하는 프리미엄 온수매트 나비엔 메이트 신제품을 출...
793,356000578,2019-08-12 06:01:07,음성인식 가능한 똑똑한 안마의자 `오레스트 OR-8000`,[기자가 체험한 중소기업 상품 톡톡-17] 최근 안마의자가 단순히 효도상품이 아닌 ...,오레스트 OR-8000의 또 다른 장점은 통상 4개 정도만 있는 마사지 볼이 8개나...,"안마의자가 단순히 효도상품이 아닌 필수품으로 각광받는 가운데, 최근 오레스트 매장에..."
1055,359634594,2019-09-11 16:24:51,"대웅제약, 현업 부서가 직접 인재 선발하는 상시 공개채용 도입",대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는...,대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는...,대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는...
1597,332732267,2019-01-30 11:00:00,"과기정통부-조달청, 'R&D 성과 공공조달 연계 위한 업무협약' 체결",과학기술정보통신부와 조달청은 30일 조달청에서 '과학기술 연구개발 성과와 공공조달 ...,과학기술정보통신부와 조달청은 30일 조달청에서 '과학기술 연구개발 성과와 공공조달 ...,과학기술정보통신부와 조달청은 30일 연구개발 성과가 국민 생활 속에서 실현되는 길을...
158,364650956,2019-10-26 10:57:00,"아이폰11 첫날 개통량, 전작보다 30% 늘었다",지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 XS·XR시리...,지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 XS·XR시리...,지난 25일 출시된 아이폰 11 시리즈의 첫날 개통량이 전작인 아이폰 xs xr시리...
1779,334116374,2019-02-14 08:46:00,"NHN엔터, 지난해 영업익 687억..'1조 클럽' 달성","NHN엔터테인먼트(181710)는 지난해 4분기 연결기준 매출 3858억원, 영업이...","NHN엔터테인먼트(181710)는 지난해 4분기 연결기준 매출 3858억원, 영업이...",nhn엔터테인먼트 ( 1817 ) 는 기존 게임사업의 체질 개선과 신사업의 확대로 ...
1388,331133400,2019-01-16 10:37:00,"네이버, 사용자 취향 분석해 쇼핑 검색결과 제공",네이버가 인공지능(AI)기반 상품 추천 시스템 에이아이템즈(AiTEMS)를 활용해 ...,네이버가 인공지능(AI)기반 상품 추천 시스템 에이아이템즈(AiTEMS)를 활용해 ...,네이버는 인공지능 ( ai ) 기반 상품 추천 시스템 에이아이템즈를 활용해 검색 결...
747,355261054,2019-08-05 14:55:17,"맵퍼스, 도로공사와 고속도로 사고 대응 MOU",전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발...,전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발...,전자지도 소프트웨어 전문기업 맵퍼스가 한국도로공사와 고속도로 사고 대응 서비스 개발...
739,355048278,2019-08-02 15:14:10,순천향대서울병원 `Best of the Best` 인정,서유성 순천향대서울병원장 순천향대학교 서울병원(원장 서유성)이 최근 보건복지부가 발...,순천향대 서울병원은 △의료질과 환자안전 △의료전달체계 △공공성 평가 영역에서 1등급...,순천향대 서울병원이 전국 337개 상급종합병원과 종합병원을 대상으로 실시한 2019...
272,368093505,2019-11-22 09:09:00,공공의대 2023년 개교하나?...공청회서 찬반 공세 예고,의료취약지와 공공보건 분야에서 복무할 의사를 양성할 국립공공보건의료대학 설립 법안에...,의료취약지와 공공보건 분야에서 복무할 의사를 양성할 국립공공보건의료대학 설립 법안에...,22일 국회 보건복지위원회는 공청회를 열어 의료취약지와 공공보건 분야에서 복무할 의...


#Rouge Score

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

In [ ]:
summary = random['summary']
result = random['result']

In [ ]:
def rouge_score(summary, result):
    summary_tokens = summary.split()
    result_tokens = result.split()

    overlap = len(set(summary_tokens) & set(result_tokens))
    recall = overlap / len(result_tokens)
    precision = overlap / len(summary_tokens)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {'rouge-1': {'f': f1_score, 'p': precision, 'r': recall}}

In [ ]:
for summary, result in zip(summary, result):
    scores = rouge_score(summary, result)
    print(f"Summary: {summary}")
    print(f"Result: {result}")
    print(f"ROUGE scores: {scores}")
    print()

Summary: 2015년 첫 선을 보인 나비엔 메이트는 온도제어 기술력을 적용해 만들어 매트 전체에 고르고 균일한 온도를 구현한다. 난방 기능도 업그레이드돼 빠른 난방 기능을 통해 기존 제품 대비 최대 40% 빠르게 온기를 전달한다. 
Result: 경동나비엔이 쾌적한 수면환경을 제공하는 프리미엄 온수매트 나비엔 메이트 신제품을 출시한다.
ROUGE scores: {'rouge-1': {'f': 0.04878048780487805, 'p': 0.03225806451612903, 'r': 0.1}}

Summary: 오레스트 OR-8000의 또 다른 장점은 통상 4개 정도만 있는 마사지 볼이 8개나 있어 더욱 효과적으로 안마를 해준다는 것이다. 또한 마사지 볼은 레일을 따라 움직이는데 보통 일자 형태인 것과 달리 OR-8000은 레일이 S자 형태로 최대한 안마받는 신체 라인을 고려했다. 
Result: 안마의자가 단순히 효도상품이 아닌 필수품으로 각광받는 가운데, 최근 오레스트 매장에는 마사지 볼이 8개나 있어 더욱 효과적으로 안마를 받을 수 있다.
ROUGE scores: {'rouge-1': {'f': 0.27586206896551724, 'p': 0.21052631578947367, 'r': 0.4}}

Summary: 대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는 '상시 공개채용' 방식을 도입해 11일부터 채용을 진행한다고 밝혔다. 이번 상시 채용의 서류 접수는 오는 24일까지 대웅제약 채용 홈페이지에서 할 수 있다. 대웅제약은 오는 19일 서울 강남구 본사의 베어홀에서 구직자들을 직접 초청하는 '채용설명회'를 진행할 예정이다. 
Result: 대웅제약은 기존의 정기 공개채용 대신 현업 부문이 필요한 때에 인재를 직접 선발하는 상시 공개 채용 방식을 도입해 새로운 채용 방식에 따라 각 직무 별 역량을 갖춘 인재를 확보할 계획이다.
ROUGE scores: {'rouge-1': {'f': 0.42666666666666